<a href="https://colab.research.google.com/github/Natsumisnooker/5001_Plotly/blob/main/Plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
ชื่อ นามสกุล รหัสนักศึกษา
ณัฐรวิภาดา จิรวรัตน์ชัย 6720422004
จรรยพร ไหลกูล 6720422022
สโรชา เคนเวียง 6720422031


In [ ]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv", sep='\t')
print(df.head())

   order_id  quantity                              item_name  \
0         1         1           Chips and Fresh Tomato Salsa   
1         1         1                                   Izze   
2         1         1                       Nantucket Nectar   
3         1         1  Chips and Tomatillo-Green Chili Salsa   
4         2         2                           Chicken Bowl   

                                  choice_description item_price  
0                                                NaN     $2.39   
1                                       [Clementine]     $3.39   
2                                            [Apple]     $3.39   
3                                                NaN     $2.39   
4  [Tomatillo-Red Chili Salsa (Hot), [Black Beans...    $16.98   


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            4622 non-null   int64 
 1   quantity            4622 non-null   int64 
 2   item_name           4622 non-null   object
 3   choice_description  3376 non-null   object
 4   item_price          4622 non-null   object
dtypes: int64(2), object(3)
memory usage: 180.7+ KB


In [2]:
import pandas as pd
import plotly.express as px

df = pd.read_csv("https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv", sep='\t')

# รวมยอด quantity ของแต่ละเมนู
top_items = df.groupby('item_name')['quantity'].sum().reset_index()

# เรียงลำดับและเลือก 10 อันดับแรก
top_10 = top_items.sort_values(by='quantity', ascending=False).head(10)

# สร้างกราฟแท่ง
fig = px.bar(top_10, x='item_name', y='quantity',
             title='10 เมนูยอดฮิต (จากจำนวนสั่งซื้อ)',
             labels={'item_name': 'เมนู', 'quantity': 'จำนวนสั่งซื้อรวม'},
             text='quantity')

fig.update_traces(textposition='outside')
fig.update_layout(xaxis_tickangle=-45)

fig.show()



In [11]:
import pandas as pd
import plotly.express as px


df = pd.read_csv("https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv", sep='\t')

df['item_price'] = df['item_price'].str.replace('$', '', regex=False).astype(float)

# สร้าง Histogram
fig = px.histogram(
    df,
    x='item_price',
    title='Distribution of Item Prices (Chipotle)',
    nbins=30, # ใช้ 30 bins เพื่อดูรายละเอียดของราคา
    marginal='box', # เพิ่ม Box Plot เพื่อแสดงสถิติสรุป (Median, Quartiles)
    color='quantity', # แบ่งสีตามจำนวนชิ้นที่สั่ง เพื่อดูว่าราคาสูงมาจากจำนวนชิ้นที่เพิ่มขึ้นหรือไม่
    opacity=0.7
)

fig.update_layout(xaxis_title='Item Price ($)', yaxis_title='Count of Records')
fig.show()

In [12]:
import pandas as pd
import plotly.express as px
import numpy as np

df = pd.read_csv("https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv", sep='\t')
df['item_price'] = df['item_price'].str.replace('$', '', regex=False).astype(float)

# รวมข้อมูลตามรายการอาหารและตัวเลือกที่ลูกค้าสั่ง
df_sunburst = df.groupby(['item_name', 'choice_description'], as_index=False).agg(
    total_quantity=('quantity', 'sum')
)

# แทนค่าว่างใน choice_description ด้วย 'No Selection'
df_sunburst['choice_description'] = df_sunburst['choice_description'].fillna('No Selection')

# สร้าง Sunburst Chart
fig = px.sunburst(
    df_sunburst,
    path=['item_name', 'choice_description'], # ลำดับชั้น: รายการหลัก -> ตัวเลือกย่อย
    values='total_quantity', # ขนาดของส่วนตามจำนวนชิ้นที่สั่งรวม
    color='total_quantity', # ใช้สีตามจำนวนชิ้นที่สั่ง
    color_continuous_scale='RdBu',
    title='Total Quantity of Items Sold by Item and Choice Description'
)

fig.update_layout(height=700)
fig.show()

In [13]:
import pandas as pd
import plotly.express as px

# แปลงราคาให้เป็น float
df['item_price'] = df['item_price'].replace('[\$,]', '', regex=True).astype(float)

# ลบรายการที่ซ้ำกันของชื่อเมนู + ราคา (เช่น Chicken Bowl ที่มีหลายราคา)
unique_items = df[['item_name', 'item_price']].drop_duplicates()

# เลือกรายการที่ราคาแพงที่สุด 10 อันดับ
top_10_prices = unique_items.sort_values(by='item_price', ascending=False).head(10)

# สร้าง scatter plot
fig = px.scatter(top_10_prices,
                 x='item_price',
                 y='item_name',
                 size='item_price',
                 color='item_price',
                 title='10 เมนูที่ราคาแพงที่สุด (ต่อคำสั่ง)',
                 labels={'item_price': 'ราคา ($)', 'item_name': 'ชื่อเมนู'},
                 text='item_price')

fig.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGrey')),
                  texttemplate='$%{text:.2f}', textposition='top center')
fig.update_layout(height=600)

fig.show()

<>:5: SyntaxWarning:

invalid escape sequence '\$'

<>:5: SyntaxWarning:

invalid escape sequence '\$'

/tmp/ipython-input-3818734493.py:5: SyntaxWarning:

invalid escape sequence '\$'



In [14]:
import pandas as pd
import plotly.express as px

df = pd.read_csv("https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv", sep='\t')
df['item_price'] = df['item_price'].str.replace('$', '', regex=False).astype(float)

# สร้าง Scatter Plot
fig = px.scatter(
    df,
    x='quantity',
    y='item_price',
    color='item_name', # แบ่งสีตามรายการอาหาร
    size='item_price', # ขนาดของจุดตามราคา
    hover_name='item_name',
    hover_data=['choice_description'],
    title='Item Price vs. Quantity Ordered',
    log_y=True # ใช้ Log Scale สำหรับแกน Y เพื่อให้เห็นจุดราคาที่สูงมากได้ชัดเจนขึ้น
)

fig.update_layout(
    xaxis_title='Quantity Ordered',
    yaxis_title='Item Price ($) (Log Scale)'
)
fig.show()

In [15]:
import pandas as pd
import plotly.express as px

# แปลง item_price เป็น float
df['item_price'] = df['item_price'].replace('[\$,]', '', regex=True).astype(float)

# คำนวณยอดขายรวม (quantity * item_price)
df['total_sales'] = df['quantity'] * df['item_price']

# รวมยอดขายตามเมนู
sales_by_item = df.groupby('item_name')['total_sales'].sum().reset_index()

# เลือก 10 อันดับเมนูที่มียอดขายสูงสุด
top_10_sales = sales_by_item.sort_values(by='total_sales', ascending=False).head(10)

# สร้าง Pie Chart
fig = px.pie(top_10_sales,
             names='item_name',
             values='total_sales',
             title='10 อันดับเมนูที่มียอดขายสูงสุด ',
             hole=0.3,  # ถ้าอยากเป็น donut chart ปรับเป็น > 0
             labels={'item_name': 'เมนู', 'total_sales': 'ยอดขายรวม ($)'},
             hover_data=['total_sales'])

fig.update_traces(textposition='inside', textinfo='percent+label')

fig.show()



<>:5: SyntaxWarning:

invalid escape sequence '\$'

<>:5: SyntaxWarning:

invalid escape sequence '\$'

/tmp/ipython-input-197248265.py:5: SyntaxWarning:

invalid escape sequence '\$'



In [19]:
import pandas as pd

df = pd.read_csv(
    "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv",
    sep="\t"
)

# ทำความสะอาดและแทนค่าว่างด้วย "Empty"
s = df['choice_description'].fillna('[]').str.strip('[]')

def split_or_empty(text):
    text = text.strip()
    if text == '':
        return ['']
    return [part.strip() for part in text.split(',')]

split = s.apply(split_or_empty)

# ให้เป็นหลายคอลัมน์และเติมช่องว่างด้วย "Empty"
wide_choices = split.apply(pd.Series).add_prefix('choice_').fillna('')

# รวมกลับเข้ากับ df (เก็บคอลัมน์เดิมไว้ก็ได้)
df_wide = pd.concat([df.drop(columns=['choice_description']), wide_choices], axis=1)
print(df_wide.head())


   order_id  quantity                              item_name item_price  \
0         1         1           Chips and Fresh Tomato Salsa     $2.39    
1         1         1                                   Izze     $3.39    
2         1         1                       Nantucket Nectar     $3.39    
3         1         1  Chips and Tomatillo-Green Chili Salsa     $2.39    
4         2         2                           Chicken Bowl    $16.98    

                          choice_0      choice_1 choice_2 choice_3  \
0                                                                    
1                       Clementine                                   
2                            Apple                                   
3                                                                    
4  Tomatillo-Red Chili Salsa (Hot)  [Black Beans     Rice   Cheese   

     choice_4 choice_5 choice_6 choice_7 choice_8 choice_9  
0                                                           
1     

In [ ]:
import pandas as pd
import plotly.express as px

df = pd.read_csv("https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv", sep='\t')

# รวมยอด quantity ของแต่ละเมนู
top_items = df.groupby('item_name')['quantity']['choice_0','choice_1','choice_2','choice_3',].sum().reset_index()

# เรียงลำดับและเลือก 10 อันดับแรก
top_10 = top_items.sort_values(by='quantity', ascending=False).head(10)
